In [ ]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.24.0.8-3.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.24.0.8-3.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
from functools import reduce

# Loading CSV Files with pandas
df_vins_data = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv")
df_population = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv")
df_cca_claims = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/CCA Claims.csv")
df_egr_cooler_claims = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/EGR Cooler Claims.csv")
df_egr_fg_293_claims = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/EGR FG 293 Claims.csv")
df_egr_sensors = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/EGR Sensors.csv")
df_engine_emissions_table1 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/Engine_Emissions_Table1.csv")
df_engine_emissions_table2 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/Engine_Emissions_Table2.csv")
df_p1075_38 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P1075_38.csv")
df_p1075_75 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P1075_75.csv")
df_p1075_77 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P1075_77.csv")
df_p1075_86 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P1075_86.csv")
df_p1075_92 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P1075_92.csv")
df_p1075_94 = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P1075_94.csv")
df_p0401_faults = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P0401_faults.csv")
df_p2457_faults = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/P2457_faults.csv")

In [ ]:
# Function to clean the data
def raw_data_cleaning(df):
    df_cleaned = df[
        (df['_C0'].notna()) & (df['_C0'] != 'NA') & (df['_C0'] != '') &
        (df['MESSAGE_ID'].notna()) & (df['MESSAGE_ID'] != '') & (df['MESSAGE_ID'] != 'NA') &
        (df['FAULT_DATE_TIME'].notna()) & (df['FAULT_DATE_TIME'] != '') & (df['FAULT_DATE_TIME'] != 'NA') &
        (df['FAULT_STATUS'].notna()) & (df['FAULT_STATUS'] != '') & (df['FAULT_STATUS'] != 'NA') &
        (df['VIN'].notna()) & (df['VIN'] != '') & (df['VIN'] != 'NA') &
        (df['VIN'].str.len() >= 17)
    ]
    return df_cleaned

# Apply the cleaning function to each dataframe
df_p1075_38_cleaned = raw_data_cleaning(df_p1075_38)
df_p1075_75_cleaned = raw_data_cleaning(df_p1075_75)
df_p1075_77_cleaned = raw_data_cleaning(df_p1075_77)
df_p1075_86_cleaned = raw_data_cleaning(df_p1075_86)
df_p1075_92_cleaned = raw_data_cleaning(df_p1075_92)
df_p1075_94_cleaned = raw_data_cleaning(df_p1075_94)
df_p0401_faults_cleaned = raw_data_cleaning(df_p0401_faults)
df_p2457_faults_cleaned = raw_data_cleaning(df_p2457_faults)

# Define feature calculation functions
def feature_1_or_2_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN, dtc_status_to_calculate_this_feature):
    df_dtc_type_and_status_for_this_VIN = df_dtc_type_for_this_VIN[df_dtc_type_for_this_VIN['FAULT_STATUS'] == dtc_status_to_calculate_this_feature]
    return len(df_dtc_type_and_status_for_this_VIN)

def feature_3_or_4_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN, dtc_type, dtc_status_to_calculate_this_feature):
    fault_date_time_format = '%Y-%m-%d %H:%M:%S' if dtc_type != "p1075_38" else '%m/%d/%Y %H:%M'
    df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] = pd.to_datetime(df_dtc_type_for_this_VIN['FAULT_DATE_TIME'], format=fault_date_time_format)
    df_dtc_type_for_this_VIN = df_dtc_type_for_this_VIN.sort_values(by="FAULT_DATE_TIME")
    
    df_dtc_type_for_this_VIN['NEXT_FAULT_DATE_TIME'] = df_dtc_type_for_this_VIN['FAULT_DATE_TIME'].shift(-1)
    df_dtc_type_for_this_VIN['TIME_DIFF'] = (df_dtc_type_for_this_VIN['NEXT_FAULT_DATE_TIME'] - df_dtc_type_for_this_VIN['FAULT_DATE_TIME']).dt.total_seconds()
    
    duration_in_this_15_days_back = df_dtc_type_for_this_VIN[
        (df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] >= start_date) &
        (df_dtc_type_for_this_VIN['NEXT_FAULT_DATE_TIME'] <= end_date) &
        (df_dtc_type_for_this_VIN['FAULT_STATUS'] == dtc_status_to_calculate_this_feature)
    ]['TIME_DIFF'].sum()
    
    return duration_in_this_15_days_back

def feature_5_or_6_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN, dtc_type, dtc_status_to_calculate_this_feature):
    fault_date_time_format = '%Y-%m-%d %H:%M:%S' if dtc_type != "p1075_38" else '%m/%d/%Y %H:%M'
    df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] = pd.to_datetime(df_dtc_type_for_this_VIN['FAULT_DATE_TIME'], format=fault_date_time_format)
    df_dtc_type_for_this_VIN = df_dtc_type_for_this_VIN.sort_values(by="FAULT_DATE_TIME")
    
    df_dtc_type_for_this_VIN['NEXT_FAULT_DATE_TIME'] = df_dtc_type_for_this_VIN['FAULT_DATE_TIME'].shift(-1)
    df_dtc_type_for_this_VIN['TIME_DIFF'] = (df_dtc_type_for_this_VIN['NEXT_FAULT_DATE_TIME'] - df_dtc_type_for_this_VIN['FAULT_DATE_TIME']).dt.total_seconds()
    
    df_filtered = df_dtc_type_for_this_VIN[
        (df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] >= start_date) &
        (df_dtc_type_for_this_VIN['NEXT_FAULT_DATE_TIME'] <= end_date) &
        (df_dtc_type_for_this_VIN['FAULT_STATUS'] == dtc_status_to_calculate_this_feature)
    ]
    
    if len(df_filtered) > 0:
        return df_filtered['TIME_DIFF'].mean()
    else:
        return 0.0

def feature_7_or_8_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN, vehicle_speed, dtc_status_to_calculate_this_feature):
    df_filtered = df_dtc_type_for_this_VIN[
        (df_dtc_type_for_this_VIN['FAULT_STATUS'] == dtc_status_to_calculate_this_feature) &
        (df_dtc_type_for_this_VIN['ROAD_SPEED_MPH'] == vehicle_speed)
    ]
    return len(df_filtered)

def if_part_is_replaced_for_this_VIN_in_this_timespan(thisVIN, start_date, end_date):
    df_cca_claims_part_replacements = df_cca_claims[
        (df_cca_claims['VIN'] == thisVIN) &
        (df_cca_claims['CLAIM_REG_DATE'] > start_date) &
        (df_cca_claims['CLAIM_REG_DATE'] < end_date) &
        (df_cca_claims['TOT_CLAIM_PAYMENT_USD'] > 1000.0)
    ]

    df_egr_cooler_claims_part_replacements = df_egr_cooler_claims[
        (df_egr_cooler_claims['VIN'] == thisVIN) &
        (df_egr_cooler_claims['CLAIM_REG_DATE'] > start_date) &
        (df_egr_cooler_claims['CLAIM_REG_DATE'] < end_date) &
        (df_egr_cooler_claims['TOT_CLAIM_PAYMENT_USD'] > 1000.0)
    ]

    df_egr_fg_293_claims_part_replacements = df_egr_fg_293_claims[
        (df_egr_fg_293_claims['VIN'] == thisVIN) &
        (df_egr_fg_293_claims['CLAIM_REG_DATE'] > start_date) &
        (df_egr_fg_293_claims['CLAIM_REG_DATE'] < end_date) &
        (df_egr_fg_293_claims['TOT_CLAIM_PAYMENT_USD'] > 1000.0)
    ]

    df_egr_sensors_claims_part_replacements = df_egr_sensors[
        (df_egr_sensors['VIN'] == thisVIN) &
        (df_egr_sensors['CLAIM_REG_DATE'] > start_date) &
        (df_egr_sensors['CLAIM_REG_DATE'] < end_date) &
        (df_egr_sensors['TOT_CLAIM_PAYMENT_USD'] > 1000.0)
    ]

    number_of_parts_replaced_for_thisVIN = len(df_cca_claims_part_replacements) + len(df_egr_cooler_claims_part_replacements) + len(df_egr_fg_293_claims_part_replacements) + len(df_egr_sensors_claims_part_replacements)
    
    if number_of_parts_replaced_for_thisVIN > 0:
        return 1
    else:
        return 0

def normalize_numeric_feature_values(statement_in_feature_name, df):
    selected_col_names_list = [col for col in df.columns if statement_in_feature_name in col]  # selects names of columns that contain specific string
    selected_cols = df[selected_col_names_list].replace(",", ".", regex=True).astype(float)
    selected_cols_norm = selected_cols.apply(lambda iterator: ((iterator - iterator.mean()) / (iterator.max() - iterator.min())).round(3))
    df[selected_cols_norm.columns] = selected_cols_norm
    return df

def move_over_calendar_and_compute_features(df_selected_features_from_population_for_this_VIN, thisVIN, new_15day_end_date, span_length, dayCount, jobID):
    print(f"A new day move on calendar: thisVIN={thisVIN}, new_15day_end_date={new_15day_end_date}, span_length={span_length}, dayCount={dayCount}")
    
    df_calculated_features_for_this_VIN_and_this_dayCount = pd.DataFrame({'VIN': [thisVIN], 'calendar_day': [dayCount]})
    
    new_15day_start_date = new_15day_end_date - timedelta(days=span_length)
    previous_15day_duration_end_date = new_15day_start_date
    previous_15day_duration_start_date = previous_15day_duration_end_date - timedelta(days=span_length)

    list_of_dtc_type = ['p1075_38', 'p1075_75', 'p1075_77', 'p1075_86', 'p1075_92', 'p1075_94', 'p0401_faults', 'p2457_faults']
    list_of_dtc_df = [df_p1075_38_cleaned, df_p1075_75_cleaned, df_p1075_77_cleaned, df_p1075_86_cleaned, df_p1075_92_cleaned, df_p1075_94_cleaned, df_p0401_faults_cleaned, df_p2457_faults_cleaned]
    
    for i, dtc_type_df in enumerate(list_of_dtc_df):
        df_dtc_type_for_this_VIN = dtc_type_df[dtc_type_df['VIN'] == thisVIN]
        
        df_dtc_type_for_this_VIN_in_first_15day_timespan = df_dtc_type_for_this_VIN[(df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] > new_15day_start_date) & (df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] < new_15day_end_date)]
        df_dtc_type_for_this_VIN_in_previous_15day_timespan = df_dtc_type_for_this_VIN[(df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] > previous_15day_duration_start_date) & (df_dtc_type_for_this_VIN['FAULT_DATE_TIME'] < previous_15day_duration_end_date)]
        
        count1 = len(df_dtc_type_for_this_VIN_in_first_15day_timespan)
        count2 = len(df_dtc_type_for_this_VIN_in_previous_15day_timespan)
        
        # Feature 1
        feature_1_value_first_15days = feature_1_or_2_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, "Y") if count1 > 0 else 0
        feature_1_value_second_15days = feature_1_or_2_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, "Y") if count2 > 0 else 0

        # Feature 2
        feature_2_value_first_15days = feature_1_or_2_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, "I") if count1 > 0 else 0
        feature_2_value_second_15days = feature_1_or_2_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, "I") if count2 > 0 else 0

        # Feature 3
        feature_3_value_first_15days = feature_3_or_4_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, list_of_dtc_type[i], "Y") if count1 > 0 else 0.0
        feature_3_value_second_15days = feature_3_or_4_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, list_of_dtc_type[i], "Y") if count2 > 0 else 0.0

        # Feature 4
        feature_4_value_first_15days = feature_3_or_4_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, list_of_dtc_type[i], "I") if count1 > 0 else 0.0
        feature_4_value_second_15days = feature_3_or_4_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, list_of_dtc_type[i], "I") if count2 > 0 else 0.0

        # Feature 5
        feature_5_value_first_15days = feature_5_or_6_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, list_of_dtc_type[i], "Y") if count1 > 0 else 0.0
        feature_5_value_second_15days = feature_5_or_6_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, list_of_dtc_type[i], "Y") if count2 > 0 else 0.0

        # Feature 6
        feature_6_value_first_15days = feature_5_or_6_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, list_of_dtc_type[i], "I") if count1 > 0 else 0.0
        feature_6_value_second_15days = feature_5_or_6_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, list_of_dtc_type[i], "I") if count2 > 0 else 0.0

        # Feature 7
        feature_7_value_first_15days = feature_7_or_8_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, 0, "Y") if count1 > 0 else 0
        feature_7_value_second_15days = feature_7_or_8_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, 0, "Y") if count2 > 0 else 0

        # Feature 8
        feature_8_value_first_15days = feature_7_or_8_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_first_15day_timespan, 0, "I") if count1 > 0 else 0
        feature_8_value_second_15days = feature_7_or_8_calculate_for_this_VIN_and_this_timespan(df_dtc_type_for_this_VIN_in_previous_15day_timespan, 0, "I") if count2 > 0 else 0

        dtc = list_of_dtc_type[i].split('_')[-1]
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_1_dtc{dtc}_first_15d'] = feature_1_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_1_dtc{dtc}_second_15d'] = feature_1_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_2_dtc{dtc}_first_15d'] = feature_2_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_2_dtc{dtc}_second_15d'] = feature_2_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_3_dtc{dtc}_first_15d'] = feature_3_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_3_dtc{dtc}_second_15d'] = feature_3_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_4_dtc{dtc}_first_15d'] = feature_4_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_4_dtc{dtc}_second_15d'] = feature_4_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_5_dtc{dtc}_first_15d'] = feature_5_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_5_dtc{dtc}_second_15d'] = feature_5_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_6_dtc{dtc}_first_15d'] = feature_6_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_6_dtc{dtc}_second_15d'] = feature_6_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_7_dtc{dtc}_first_15d'] = feature_7_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_7_dtc{dtc}_second_15d'] = feature_7_value_second_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_8_dtc{dtc}_first_15d'] = feature_8_value_first_15days
        df_calculated_features_for_this_VIN_and_this_dayCount[f'feature_8_dtc{dtc}_second_15d'] = feature_8_value_second_15days
    
    # Check for part replacements in the two 15-day periods
    df_calculated_features_for_this_VIN_and_this_dayCount['if_parts_replaced_in_first_15days'] = if_part_is_replaced_for_this_VIN_in_this_timespan(thisVIN, new_15day_start_date, new_15day_end_date)
    df_calculated_features_for_this_VIN_and_this_dayCount['if_parts_replaced_in_second_15days'] = if_part_is_replaced_for_this_VIN_in_this_timespan(thisVIN, previous_15day_duration_start_date, previous_15day_duration_end_date)

    # Update the VINs data file with the new day count
    df_vins_data.loc[df_vins_data['VIN'] == thisVIN, 'TOTAL_ROWS'] = dayCount

    # Aggregate all selected features from the population with the calculated feature values for this VIN and write it to resultedData.csv
    df_features_for_this_VIN_and_this_dayCount = pd.merge(df_selected_features_from_population_for_this_VIN, df_calculated_features_for_this_VIN_and_this_dayCount, on="VIN")
    df_features_for_this_VIN_and_this_dayCount.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/resultedData.csv', index=False, mode='a', header=False)
    return


In [ ]:
# Execution block
if len(sys.argv) > 1:
    thisVIN = sys.argv[1]
    the_calculator_jobID_for_thisVIN = sys.argv[2]
    
    print(f"Current VIN: {thisVIN}")
    
    duration_end_date = '2021-12-31'
    day_delta = timedelta(days=1)
    split_date = duration_end_date.split('-')

    end_date = date(int(split_date[0]), int(split_date[1]), int(split_date[2]))
    start_date = end_date - timedelta(days=1826)

    span_length = 15

    # Prepare data for this VIN
    df_new_VIN = pd.DataFrame({'VIN': [thisVIN], 'TOTAL_ROWS': [0]})
    
    df_filtered_population_for_this_VIN = df_population[df_population['VIN'] == thisVIN]
    selected_features_from_population_for_this_VIN = ['VIN', 'ENGINE_SIZE', 'ENGINE_HP', 'VEH_TYPE'] + [s for s in df_filtered_population_for_this_VIN.columns if 'KOLA' in s]
    df_selected_features_from_population_for_this_VIN = df_filtered_population_for_this_VIN[selected_features_from_population_for_this_VIN]
    
    if not df_selected_features_from_population_for_this_VIN.empty:
        how_many_month = int((end_date - start_date).days / span_length)
        
        if how_many_month == 0:
            remaining_days = int((end_date - start_date).days)
            Parallel(n_jobs=5, prefer="threads", batch_size=5)(
                delayed(move_over_calendar_and_compute_features)(
                    df_selected_features_from_population_for_this_VIN, thisVIN, end_date, span_length, day_count_in_month, the_calculator_jobID_for_thisVIN
                ) for day_count_in_month in range(remaining_days)
            )
        else:
            for number_of_monthes_in_time_duration in range(how_many_month):
                if number_of_monthes_in_time_duration < how_many_month:
                    Parallel(n_jobs=5, prefer="threads", batch_size=5)(
                        delayed(move_over_calendar_and_compute_features)(
                            df_selected_features_from_population_for_this_VIN, thisVIN, end_date, span_length, span_length * number_of_monthes_in_time_duration + day_count_in_month, the_calculator_jobID_for_thisVIN
                        ) for day_count_in_month in range(span_length)
                    )
                else:
                    remaining_days = int((end_date - start_date).days) - span_length * number_of_monthes_in_time_duration
                    Parallel(n_jobs=5, prefer="threads", batch_size=5)(
                        delayed(move_over_calendar_and_compute_features)(
                            df_selected_features_from_population_for_this_VIN, thisVIN, end_date, span_length, span_length * number_of_monthes_in_time_duration + day_count_in_month, the_calculator_jobID_for_thisVIN
                        ) for day_count_in_month in range(remaining_days)
                    )
else:
    print("The VIN argument is missing for feature calculations.")
